In [2]:
import mne
import os
import glob
import numpy as np
import pandas as pd
from scipy.io import savemat, loadmat
import matplotlib.pyplot as plt
import pickle

import sys
sys.path.append('../utils')
from pp_utils import *
from plot_utils import *


In [ ]:
def create_mapchange_df(first_keystrokes,  all_keystrokes):
    """ 
    Makes a df with the times and indices for first and other keystrokes
        for first keystrokes, further adds the number of keystrokes in the previous map
    returns: mapchange_df
    """

    mapchange_df = pd.DataFrame(columns = ['keystroke_idx', 'time', 'type', 'prev_keystrokes'])
    mapchange_df['time'] = all_keystrokes[:,0]

    mapchange_df['keystroke_idx'] = mapchange_df.index
    mapchange_df['type'] = mapchange_df['time'].apply(
            lambda x: 'first' if x in first_keystrokes[:, 0] 
            else 'other'
    )

    # Find the indices of rows where type is 'first'
    first_indices = mapchange_df.index[mapchange_df['type'] == 'first'].tolist()

    # Iterate over these indices and count 'other' keystrokes
    for i in range(len(first_indices)):
        current_index = first_indices[i]
        if i == 0:
            # For the first 'first', there are no previous rows to count
            mapchange_df.at[current_index, 'prev_keystrokes'] = 0
        else:
            previous_index = first_indices[i - 1]
            # Count 'other' keystrokes between the two 'first' rows
            count_others = mapchange_df.loc[previous_index+1:current_index-1, 'type'].value_counts().get('other', 0)
            mapchange_df.at[current_index, 'prev_keystrokes'] = count_others

    return mapchange_df

In [44]:
subjects_to_process = ['01']
periods = ['pre', 'post']
keystroke_trigs = 'audio'

pp_dir = "/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/data_preprocessed_30Hz" 
mapchange_dir = './mapchange_csv/'
if not os.path.exists(mapchange_dir):
    os.mkdir(mapchange_dir)

for folder in sorted(os.listdir(pp_dir)):
    if folder not in subjects_to_process:
        continue

    print('\nPROCESSING SUBJECT ', folder)
    sub_pp_dir = os.path.join(pp_dir, folder)
    for period in periods:
        data_path = glob.glob(os.path.join(sub_pp_dir, f'eeg_error_{period}_??.mat'))[0]

        subject_ID = data_path.split('.')[0][-2:]

        print('Opening', data_path.split('/')[-1])
        

        #--------------------------------------------
        #               LOAD FILE AND EVENTS
        #--------------------------------------------
        data = loadmat(data_path)
        events_sv = data['events']

        events_arr = make_raw_events(events_sv)

        if keystroke_trigs == 'MIDI':
            t_keystrokes = clean_triggers(events_arr[events_arr[:, 2]==6])
        else:
            t_keystrokes = clean_triggers(events_arr[events_arr[:, 2]==2])

        first_keystrokes = mapchange_keystrokes(t_modeswitch = t_modeswitch, t_keystroke=t_keystrokes)
        #other_keystrokes = withinmap_keystrokes(t_keystrokes, first_keystrokes)   
        
        mapchange_df = create_mapchange_df(first_keystrokes, t_keystrokes)

        mapchange_df.to_csv(os.path.join(mapchange_dir, f'mapchange_keystrokes_{period}_{subject_ID}.csv'))



PROCESSING SUBJECT  01
Opening eeg_error_pre_01.mat
Opening eeg_error_post_01.mat
